In [14]:
import os
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama as OllamaLLM
import streamlit as st



In [15]:
# 加载本地PDF文档
loader = PyPDFLoader("123.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)


In [18]:

# 创建 OllamaEmbeddings 和向量存储
embeddings = OllamaEmbeddings(model="gemma:2b")
docsearch = Chroma.from_documents(texts, embeddings)

# 创建问答链
qa_chain = RetrievalQA.from_chain_type(
    llm=OllamaLLM(model="gemma:2b"),
    chain_type="stuff",
    retriever=docsearch.as_retriever()
)

In [19]:
# Streamlit 应用程序
def main():
    st.title("Local PDF Document QA")

    # 对话历史记录
    if "chat_history" not in st.session_state:
        st.session_state.chat_history = []

    # 用户输入查询
    query = st.text_input("Enter your question about the document:")

    if query:
        # 执行问答
        result = qa_chain.run(query)

        # 将查询和答案添加到对话历史记录中
        st.session_state.chat_history.append({"query": query, "answer": result})

    # 显示对话历史记录
    for chat in st.session_state.chat_history:
        st.write(f"Question: {chat['query']}")
        st.write(f"Answer: {chat['answer']}")
        st.write("---")


if __name__ == "__main__":
    main()

2024-04-20 11:33:59.875 
  command:

    streamlit run C:\Software\anaconda\conda\envs\tf20\lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2024-04-20 11:33:59.878 Session state does not function when running a script without `streamlit run`


AttributeError: st.session_state has no attribute "chat_history". Did you forget to initialize it? More info: https://docs.streamlit.io/library/advanced-features/session-state#initialization

# 2

In [22]:
from langchain_community.llms import Ollama as OllamaLLM


from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# mistral = Mistral(model_path="~/mistral-models/mistral-base")
mistral = OllamaLLM(model="gemma:2b")


template = """
请总结以下内容，生成一个JSON格式的世界杯冠军队伍列表:
{text}

列表应该包括以下健：
year - 举办年份
host - 主办国
champion - 冠军队
ranner_up - 亚军队

JSON:
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)


llm_chain = LLMChain(prompt=prompt, llm=mistral)
input_data = {
        "text": "2022年世界杯足球赛是第22届世界杯足球赛,于2022年11月20日至12月18日在卡塔尔举行。这是首次在阿拉伯国家举行,也是继2002年韩日世界杯后第二次在亚洲举行。法国队在决赛中经过点球大战击败阿根廷队,夺得队史第三个世界冠军,追平巴西队和意大利队,成为夺得世界杯次数第二多的球队。"
    }

# 使用invoke方法，并传递input_data作为参数
result= llm_chain.run(input_data)
print(result)




{
  "year": 2022,
  "host": "卡塔尔",
  "champion": "法国",
  "runner_up": "阿根廷"
}


# 3

In [23]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

mistral = OllamaLLM(model="gemma:2b")

template = """
你是一位经验丰富的数据分析师,请根据以下的数据描述,设计一个合适的JSON数据结构来存储这些数据。

数据描述:
一家电商公司希望分析其销售数据,以更好地了解业务情况。他们提供了以下信息:
- 每笔交易都有一个唯一的交易ID
- 每笔交易都有一个关联的客户ID
- 每个客户都有姓名、邮箱和电话等联系信息
- 每笔交易包含一个或多个商品,每个商品都有名称、类别和价格
- 交易还包含总金额、交易日期和状态(已完成、已取消、退款等)

请根据这些信息,设计一个JSON格式,尽可能清晰地组织这些数据。
你给出的JSON结构应该在 ```json 和 ``` 之间。
除了JSON结构外,请再给出2-3条示例数据。

JSON结构和示例数据:
```json
{json_schema}
"""

prompt = PromptTemplate(
input_variables=["json_schema"],
template=template
)

llm_chain = LLMChain(prompt=prompt, llm=mistral)


result = llm_chain.invoke(input={"json_schema": ""})
print(result["text"])

```json
{
  "transactions": [
    {
      "transactionId": 12345,
      "customerId": 67890,
      "customerName": "John Smith",
      "customerEmail": "john.smith@example.com",
      "customerPhone": "123-456-7890",
      "products": [
        {
          "productId": 1011,
          "productName": "Shirt",
          "productCategory": "Clothing",
          "productPrice": 29.99
        },
        {
          "productId": 1012,
          "productName": "Shoes",
          "productCategory": "Clothing",
          "productPrice": 49.99
        }
      ],
      "totalAmount": 79.97,
      "transactionDate": "2023-03-01",
      "status": "Completed"
    },
    ... // 其他交易数据
  ]
}
```

**示例数据:**

1. 交易ID: 12345, 客户ID: 67890, 顾客姓名: John Smith, 顾客邮箱: john.smith@example.com, 顾客电话: 123-456-7890, 商品: Shirt (产品ID: 1011, 产品名称: Shirt, 产品类别: Clothing, 产品价格: 29.99)，总金额: 79.97, 状态: 已完成。
2. 交易ID: 67890, 客户ID: 10112, 顾客姓名: Jane Doe, 顾客邮箱: jane.doe@example.com, 顾客电话: 456-789-0123, 商品: Shoes (产品ID: 1012, 

# 4

In [25]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

mistral = OllamaLLM(model="gemma:2b")

template = """
给定以下文本:
{text}

请检查上述文本是否包含制作美食的步骤说明。
如果包含,请用markdown格式列出这些步骤,其中每个步骤都以"1. "开头,并在步骤前添加一个标题"制作步骤:"。
如果文本中未提供制作步骤,请直接回答"文本中未提供制作步骤"。
"""

prompt = PromptTemplate(
    input_variables=["text"],
    template=template
)

llm_chain = LLMChain(prompt=prompt, llm=mistral)

# 示例1:包含制作步骤的文本
text1 = """
麻婆豆腐制作步骤:
1. 准备好所有材料,豆腐切成小块。
2. 油热后加入花椒,煸炒15秒至有香味。加入酱油豆瓣酱煸炒1分钟,使豆瓣酱的红油充分析出。
3. 加入蒜末、姜末、葱末、肉末,翻炒至肉末变色。
4. 加入辣椒段、郫县豆瓣酱、味精、糖和水,烧开。
5. 放入豆腐块,大火煮开,转小火煮5分钟。
6. 大火收汁,淋入水淀粉勾芡,加入花椒油和香油,即可出锅。 
"""

# 示例2:不包含制作步骤的文本
text2 = """
麻婆豆腐是四川省地方传统名菜之一,属于川菜系。据说起源与清代木婆寺的一个叫陈春富的僧人。他创制了一种豆腐烹饪方法,即将豆腐切成小块,用辣椒和花椒等作料炒制而成。陈春富死后,其徒弟将木婆豆腐制作方法传了出去,经过不断改进,成为现在的麻婆豆腐。
"""

result1 = llm_chain.run(**{"text": text1})
print(result1)

result2 = llm_chain.run(**{"text": text2})
print(result2)

制作麻婆豆腐步骤如下：

**制作步骤:**

1. 准备好所有材料,豆腐切成小块。
2. 油热后加入花椒,seamnă炒15秒至有香味。加入酱油豆瓣酱seamnă烧1分钟,使豆瓣酱的红油充分析出。
3. 加入蒜末、姜末、葱末、肉末,翻炒至肉末变色。
4. 加入辣椒段、四川豆瓣酱、味精、糖和水,烧开。
5. 放入豆腐块,大火煮开,转小火煮5分钟。
6. 大火收汁,淋入水淀粉勾芡,加入花椒油和香油,即可出锅。
**制作步骤:**

1. 麻婆豆腐的制作方法是将豆腐切成小块，用辣椒和花椒等作料炒制而成。

**答案:**
文本中包含制作麻婆豆腐的步骤说明，其中每个步骤都以“1.”开头。


# 4

In [28]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

mistral = OllamaLLM(model="gemma:2b")

template = """
你是一位助理,需要根据用户提供的信息来生成一份个人简介。但是,你需要先检查用户是否提供了足够的信息。一份完整的个人简介应该包含以下

内容:
姓名
年龄
职业
教育背景
兴趣爱好
如果用户提供的信息不完整,请列出缺失的项目,并友好地提示用户提供更多信息。如果信息完整,则根据提供的信息生成一份简短的个人简介。

用户提供的信息如下:
{user_input}

助理:

"""

prompt = PromptTemplate(
input_variables=["user_input"],
template=template
)

llm_chain = LLMChain(prompt=prompt, llm=mistral)

user_input = """
姓名:张伟
年龄:28岁
教育背景:北京大学计算机科学专业硕士
"""

# result = llm_chain.run(input={"user_input": user_input})
# print(result["text"])

result = llm_chain.invoke(input={"user_input": user_input})
print(result["text"])

**缺失的信息：**

* 职业
* 兴趣爱好


**简短个人简介：**

张伟是一个28岁的计算机科学专业硕士毕业的软件工程师。他爱好阅读和编程，喜欢探索新的技术和工具。


# 5

In [30]:
from langchain_community.llms import Ollama as OllamaLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

mistral = OllamaLLM(model="")

template = """
你是一位聊天机器人,你的任务是根据用户提供的少量例子,学习如何对不同类型的问题给出幽默、风趣的回答。
以下是用户提供的两个例子:
"context": 用户:我今天感觉很不开心,你能安慰我一下吗? 
"question":不开心就像坐过山车,现在虽然在低谷,但很快就会冲上高峰!想想你拥有的美好事物,比如我这个永远支持你的超级无敌大帅机器人朋友!

"context": 用户:我最近胖了好多,感觉自己像个气球一样。
"question":别灰心,你现在不是气球,顶多是个可爱的小馒头!记住,减肥是一段旅程,而不是一天的工作。坚持下去,终有一天,你会从馒头变成一个苗条的花棍!我会一直在旁边为你加油的!

用户提供的信息如下:
{user_input}

question: 

"""

prompt = PromptTemplate(
input_variables=["user_input"],
template=template
)

llm_chain = LLMChain(prompt=prompt, llm=mistral)

user_input = """
我最近添了一只小猫,但它老是半夜把我吵醒,我该怎么办?
"""

# result = llm_chain.run(input={"user_input": user_input})
# print(result["text"])

result = llm_chain.invoke(input={"user_input": user_input})
print(result["text"])

**第一条例子**

"context": 用户:我今天感觉很不开心,你能安慰我一下吗? 

"question": 

> 哎呀，别生气啦，这只猫可是我的最宝贝了，它总是在我睡前吵吵闹闹闹，而且总是把我的床铺得像一个毛茸茸的小睡衣。我可是一个睡眠需求超级大的人，根本不适合夜间活动。😭😭

**第二条例子**

"context": 用户:我最近胖了好多,感觉自己像个气球一样。

"question": 

> 别灰心，你现在不是气球，顶多是个可爱的小馒头！记住，减肥是一段旅程，而不是一天的工作。坚持下去,终有一天,你会从馒头变成一个苗条的花棍!我会一直在旁边为你加油的，你只要坚持不懈，别放弃哦！
